### prerequisites:
- get precompiled opencv training binaries from https://sourceforge.net/projects/opencvlibrary/files/3.4.16/opencv-3.4.16-vc14_vc15.exe/download
    - in extracted folder \opencv\build\x64\vc15\bin
        - opencv_createsamples.exe
        - opencv_traincascade.exe
- positive images
- negative images

### docs & info:
- https://docs.opencv.org/4.10.0/dc/d88/tutorial_traincascade.html
- https://www.youtube.com/watch?v=XrCAvs9AePM


In [3]:
# utility to run a command and print the output in real time
# https://stackoverflow.com/a/52576723/13823404
from subprocess import Popen, PIPE, STDOUT
from IPython.core.magic import register_line_magic
@register_line_magic
def runrealcmd(command):
    process = Popen(command, stdout=PIPE, shell=True, stderr=STDOUT, bufsize=1, close_fds=True)
    for line in iter(process.stdout.readline, b''):
        print(line.rstrip().decode('utf-8'))
    process.stdout.close()
    process.wait()

In [1]:
# generate the file with negative images
import os
with open('neg.txt', 'w') as f:
    folders = ['negative/own', 'negative/croccy']
    for folder in folders:
        for filename in os.listdir(folder):
            f.write(f'{folder}/{filename}\n')

In [12]:
# annotate the positive images
input_path = "D:/School/UAntwerpen/Honours/Salamander/salamander-tracking/input/Universiteit Antwerpen/Rune De Coninck - Foto's"
for year in ['2018', '2019', '2021', '2022', '2024']:
    runrealcmd(f'opencv_annotation.exe --annotations=annotations/anno_{year}.txt -i="{input_path}/{year}"')

* mark rectangles with the left mouse button,
* press 'c' to accept a selection,
* press 'd' to delete the latest selection,
* press 'n' to proceed with next image,
* press 'esc' to stop.


In [14]:
# merge the annotations
with open('merged_annotations.txt', 'w') as f:
    # loop over all the filenames
    for filename in os.listdir('annotations'):
        if filename.startswith('anno_'):
            with open(f'annotations/{filename}', 'r') as f2:
                f.write(f2.read())

In [42]:
# generate the file with positive samples
runrealcmd("bins\opencv_createsamples.exe -vec positive_samples.dat -info merged_annotations.txt -num 570 -w 24 -h 24")

Info file name: merged_annotations.txt
Img file name: (NULL)
Vec file name: positive_samples.dat
BG  file name: (NULL)
Num: 570
BG color: 0
BG threshold: 80
Invert: FALSE
Max intensity deviation: 40
Max x angle: 1.1
Max y angle: 1.1
Max z angle: 0.5
Show samples: FALSE
Width: 24
Height: 24
Max Scale: -1
RNG Seed: 12345
Create training samples from images collection...
Done. Created 570 samples


In [53]:
# do the actual training
# clear previous cascade folder
runrealcmd("rmdir /s /q cascade")
runrealcmd("mkdir cascade")
runrealcmd("bins\opencv_traincascade.exe -data cascade/ -vec positive_samples.dat -bg neg.txt -numPos 570 -numNeg 285 -numStages 4 -w 24 -h 24 -precalcValBufSize 4096 -precalcIdxBufSize 4096 -featureType HAAR -minHitRate 0.999 -maxFalseAlarmRate 0.1 -mode ALL")

PARAMETERS:
cascadeDirName: cascade/
vecFileName: positive_samples.dat
bgFileName: neg.txt
numPos: 570
numNeg: 285
numStages: 4
precalcValBufSize[Mb] : 4096
precalcIdxBufSize[Mb] : 4096
acceptanceRatioBreakValue : -1
stageType: BOOST
featureType: HAAR
sampleWidth: 24
sampleHeight: 24
boostType: GAB
minHitRate: 0.999
maxFalseAlarmRate: 0.1
weightTrimRate: 0.95
maxDepth: 1
maxWeakCount: 100
mode: ALL
Number of unique features given windowSize [24,24] : 261600

===== TRAINING 0-stage =====
<BEGIN
POS count : consumed   570 : 570
NEG count : acceptanceRatio    285 : 1
Precalculation time: 0.83
+----+---------+---------+
|  N |    HR   |    FA   |
+----+---------+---------+
|   1|        1|        1|
+----+---------+---------+
|   2|        1|        1|
+----+---------+---------+
|   3|        1| 0.105263|
+----+---------+---------+
|   4|        1|0.0280702|
+----+---------+---------+
END>
Training until now has taken 0 days 0 hours 0 minutes 4 seconds.

===== TRAINING 1-stage =====
<BEGIN